In [1]:
# -*- coding: utf-8 -*-
import numpy as np


def activation(x):
    return 1 / (1 + np.exp(-x))


def sigma_derivative(x):
    return x * (1 - x)


# X - это матрица, где столбцы это признаки,
# а строки это объекты выборки.
X = np.array([[0, 0, 1],
              [0.3, 1, 0],
              [1, 0.3, 1],
              [0.6, 0.2, 0],
              [0.6, 0.2, 1]])

# X = np.array([[0, 0, 1],
#               [0.3, 1, 0],
#               [1, 0.3, 0],
#               [0.6, 0.2, 1],
#               [0.6, 0.2, 1]])

# X - это матрица, где столбцы это признаки
# (в данном случае один целевой признак),
# а строки это объекты выборки.
y = np.array([[0],
              [1],
              [1],
              [0],
              [0]])

np.random.seed(4)

# Нотация слоев - L1, L2, L3.
# В каждом слое есть n1, n2, n3 нейронов.
# Строки в матрицах индексируются как i, столбцы как j.

# W_1_2 - это матрица весов между первым и вторым слоем.
# Строки определяют левый нейрон (т.е. нейрон первого слоя) и соответственно количество строк равно n1.
# Столбцы определяют правый нейрон (т.е. нейрон второго слоя) и соответственно количество столбцов равно n2.
# На пересечении i-ой строки и j-го столбца получаем ячейку с конкретным весом, который
# связывает i-ый левый нейрон (слоя L1) и j-ый правый нейрон (слоя L2)
W_1_2 = 2 * np.random.random((3, 5)) - 1

# W_2_3 - это матрица весов между вторым и третьим слоем. Все остальное как в W_1_2
W_2_3 = 2 * np.random.random((5, 1)) - 1

speed = 1.1

for j in range(100000):
    # lo, l1, l2 - матрицы определенного слоя сети. Каждая строка матрицы это реакция на
    # i-ый объект входа. Каждая колонка матрицы это реакиця j-го нейрона соответствующего слоя на разные входные образы.
    # На пересечении i-ой строки и j-го столбца получаем ячейку с конкретной реакцией j-го нейрона на конкретный вход.


    # Первый слой нейронов полностю принимает значения входа Х (т.е. все реакции единичные).
    l1 = X

    # Второй слой нейронов расчитывается как функция активации по каждому элементу матрицы U
    # По сути l2 это матрица 4 на 4, где в каждой ячейке результат актиации нейрона второго слоя для всех 4-ех входных образов
    # Детально:
    # матрица U = np.dot(l0, W_0_1) является результатом
    # матричного произведения выходов нейронов предыдущего слоя на веса между 1 и 2 слоем.
    # Строки матрицы U отвечают за конкретный входной образ (объект).
    # Столбцы матрицы U отвечают за нейроны правого слоя (L2).
    # На пересечении i-ой строки и j-го столбца получаем ячейку с конкретной взвешенной суммой
    # для i-го входного образа и j-го нейрона слоя (l2). Иными словами для каждого нейрона слоя L2 и для каждого входа
    # считается U = W1X1 + W2X2 + W3X3 (поскольку входной нейрон содержит 3 нейрона).
    l2 = activation(np.dot(l1, W_1_2))

    # тоже самое проделываем для Третьего слоя
    # По сути l3 это матрица 4 на 1, где в каждой ячейке результат актиации нейрона третьего слоя (а он один)
    #  для всех 4-ех входных образов
    l3 = activation(np.dot(l2, W_2_3))

    # расчитывает ошибку на выходе
    l3_error = y - l3

    # расчитывает модуль средней ошибки
    if (j % 10000) == 0:
        print("Error:" + str(np.mean(np.abs(l3_error))))

    # sigma - есть локальный градиент ошибки

    # l3_sigma расчитывается как ошибка выхода всей сети на производную функции активации всех нейронов L3.
    # На пересечении i-ой строки и j-го столбца получаем ячейку с конкретной сигмой
    # для i-го входного образа и j-го нейрона слоя (l3). Иными словами для каждого нейрона слоя L3
    # и для каждого входа расчитывается производная по функции активации умноженная на ошибку.
    # То есть l3_sigma будет матрица 4 на 1 (поскольку в L3 только один нейрон).
    l3_sigma = l3_error * sigma_derivative(l3)

    # Ошибка L2 слоя оценивается через взвешенную сигму слоя L3 по весам между L2 и L3
    # Поскольку l3_sigma это матрица 4 на 1 и матрица W_2_3 4 на 1, то последнюю матрицу
    # надо Транспонировать, чтобы выполнить правило умножения матриц и взвесить элементы l3_sigma
    # по элементам W_2_3.
    # Тогда итоговая матрица l2_error будет 4 на 4, где на пересечении i-ой строки и j-го столбца
    # будет ячейка с конкретной ошибкой j-го нейрона слоя L2 для i-ого входного образа.
    l2_error = l3_sigma.dot(W_2_3.T)

    # l2_sigma расчитывается как ошибка слоя L2 на производную функции активации всех нейронов L2.
    # На пересечении i-ой строки и j-го столбца получаем ячейку с конкретной сигмой
    # для i-го входного образа и j-го нейрона слоя (L2). Иными словами для каждого нейрона слоя L2
    # и для каждого входа расчитывается производная по функции активации умноженная на ошибку.
    # То есть l2_sigma будет матрица 4 на 4 (поскольку в L2 четыре нейрона).
    l2_sigma = l2_error * sigma_derivative(l2)

    # обновляем веса

    # l2 это матрица 4 на 4, где в каждой ячейке результат актиации нейрона второго слоя для
    # всех 4-ех входных образов (по строкам).
    # А l3_sigma это матрица 4 на 1, где в каждой строке локальный градиент ошибки для 4-ех входных образов.
    # Чтобы взвесить столбец матрицы l2 по локальному градиенту (l3_sigma) нужно транспонировать матрицу l2, чтобы
    # результат активации каждого нейрона в слое L2 по каждому входному образу вытянулся в строку.
    # Тогда соответствующая строка умножится на столбик l3_sigma.
    # Заметьте, что транспонирование l3_sigma не даст результата, так как тогда в каждом столбце
    # l3_sigma будет по одному значению, а в каждой строке l2 по 4 значения, а значит перемножение матриц не пройдет.
    W_2_3 += speed * l2.T.dot(l3_sigma)

    # аналогично W_2_3
    W_1_2 += speed * l1.T.dot(l2_sigma)



# Прямое распространение для тестовых данных
X_test = np.array([[0, 0, 0],
                   [0.6, 0.8, 1],
                   [0.6, 0.6, 1],
                   [1, 1, 0],
                   [0.1, 0.1, 0],
                   [0.2, 0.2, 1]])

# Y_test должен получиться [0, 1, 1, 1, 0, 0]

l1 = X_test
l2 = activation(np.dot(l1, W_1_2))
l3 = activation(np.dot(l2, W_2_3))
print(l3)


Error:0.4692677031181711
Error:0.007389464275819224
Error:0.0049285194301067325
Error:0.00392037855516743
Error:0.0033414788210913435
Error:0.002955589718519708
Error:0.0026754446457782103
Error:0.0024604603544171283
Error:0.002288910989589701
Error:0.0021479944518608463
[[1.67085900e-05]
 [9.86968263e-01]
 [7.75451681e-01]
 [2.71376581e-01]
 [1.55096932e-03]
 [2.06919456e-05]]


In [2]:
"""
=====================================
Visualization of MLP weights on MNIST
=====================================

Sometimes looking at the learned coefficients of a neural network can provide
insight into the learning behavior. For example if weights look unstructured,
maybe some were not used at all, or if very large coefficients exist, maybe
regularization was too low or the learning rate too high.

This example shows how to plot some of the first layer weights in a
MLPClassifier trained on the MNIST dataset.

The input data consists of 28x28 pixel handwritten digits, leading to 784
features in the dataset. Therefore the first layer weight matrix have the shape
(784, hidden_layer_sizes[0]).  We can therefore visualize a single column of
the weight matrix as a 28x28 pixel image.

To make the example run faster, we use very few hidden units, and train only
for a very short time. Training longer would result in weights with a much
smoother spatial appearance.
"""

print(__doc__)

import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.neural_network import MLPClassifier

mnist = fetch_mldata("MNIST original")
# rescale the data, use the traditional train/test split
X, y = mnist.data / 255., mnist.target
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

#mlp = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=400, alpha=1e-4,
#                    solver='sgd', verbose=10, tol=1e-4, random_state=1)
mlp = MLPClassifier(hidden_layer_sizes=(50,40,30), max_iter=10, alpha=1e-4,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.1)

mlp.fit(X_train, y_train)
print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

fig, axes = plt.subplots(4, 4)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = mlp.coefs_[0].min(), mlp.coefs_[0].max()
for coef, ax in zip(mlp.coefs_[0].T, axes.ravel()):
    ax.matshow(coef.reshape(28, 28), cmap=plt.cm.gray, vmin=.5 * vmin,
               vmax=.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())

plt.show()

for test in range(5):
    img_digits = [X_test[random.randint(0, 9000)] for i in range(4)]
    res = mlp.predict_proba(img_digits)

    i = 0
    fig, ax = plt.subplots(nrows=2, ncols=2)
    for row in ax:
        for col in row:
            image = np.array(img_digits[i], dtype='float')
            pixels = image.reshape((28, 28))
            col.imshow(pixels, cmap='gray')
            m = max(res[i])
            img_num = [ind for ind, j in enumerate(res[i]) if j == m]
            title = 'num {0}: {1:.2f}'.format(img_num, m)
            print(title)
            col.set_title(title)
            i += 1
    plt.show()



Visualization of MLP weights on MNIST

Sometimes looking at the learned coefficients of a neural network can provide
insight into the learning behavior. For example if weights look unstructured,
maybe some were not used at all, or if very large coefficients exist, maybe
regularization was too low or the learning rate too high.

This example shows how to plot some of the first layer weights in a
MLPClassifier trained on the MNIST dataset.

The input data consists of 28x28 pixel handwritten digits, leading to 784
features in the dataset. Therefore the first layer weight matrix have the shape
(784, hidden_layer_sizes[0]).  We can therefore visualize a single column of
the weight matrix as a 28x28 pixel image.

To make the example run faster, we use very few hidden units, and train only
for a very short time. Training longer would result in weights with a much
smoother spatial appearance.



/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function fetch_mldata is deprecated; fetch_mldata was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:85: DeprecationWarning: Function mldata_filename is deprecated; mldata_filename was deprecated in version 0.20 and will be removed in version 0.22. Please use fetch_openml.
  warnings.warn(msg, category=DeprecationWarning)


URLError: <urlopen error [Errno 11] Resource temporarily unavailable>